# Extract $31^3$ samples from the HR4 catalog at $z=0$

In [1]:
# generate edges 
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
import gc

# plot settings
plt.rc('font', family='serif') 
plt.rc('font', serif='Times New Roman') 
plt.rcParams.update({'font.size': 16})
plt.rcParams['mathtext.fontset'] = 'stix'

In [2]:
from pyspark import SparkContext   
from pyspark.sql import SQLContext

#sc = SparkContext(master='local[3]', appName='calgraph')
sqlsc = SQLContext(sc)
#sc.setCheckpointDir("./checkpoints")
#sc.setCheckpointDir("hdfs://localhost:8020/myhdfs/spark/checkpoints")
sc.setCheckpointDir("hdfs://master:54310/tmp/spark/checkpoints")

import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark import Row
from pyspark.sql.window import Window as W

In [3]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

* ###  Read the halo csv file to the dataframe `halodf`

In [4]:
halo_schema = T.StructType([\
                            T.StructField('haloid', T.IntegerType(), False),\
                            T.StructField('px', T.FloatType(), False),\
                            T.StructField('py', T.FloatType(), False),\
                            T.StructField('pz', T.FloatType(), False),\
                            T.StructField('halomass', T.FloatType(), False),\
                          ])

In [5]:
halodf = sqlsc.read.csv("hdfs://master:54310/data/spark/hr4/hr4z0.csv",\
                        header=True, schema = halo_schema)

In [6]:
sys.getsizeof(halodf)

64

In [7]:
halodf.show(3,truncate=True)

+------+---------+---------+---------+-------------+
|haloid|       px|       py|       pz|     halomass|
+------+---------+---------+---------+-------------+
|     0|106.23875|2820.2603|310.53067|3.29161999E14|
|     1|1015.0091| 3070.103|2687.5447|   5.79631E14|
|     2|1150.7571| 656.3275|195.96417| 7.4869997E14|
+------+---------+---------+---------+-------------+
only showing top 3 rows



In [8]:
%time halodf.select('haloid','px','py','pz').describe().show()

+-------+--------------------+------------------+------------------+------------------+
|summary|              haloid|                px|                py|                pz|
+-------+--------------------+------------------+------------------+------------------+
|  count|           362123180|         362123180|         362123180|         362123180|
|   mean|1.8106158950000003E8|1574.9487699428887|1574.5065974890745|1575.7579631810697|
| stddev|1.0453595787073924E8| 909.2939271893584|  909.506639813124| 909.4171871354729|
|    min|                   0|         -3.628511|         -3.603208|          0.163826|
|    max|           362123179|         3149.9463|         3149.9321|         3154.1765|
+-------+--------------------+------------------+------------------+------------------+

CPU times: user 8.9 ms, sys: 4.51 ms, total: 13.4 ms
Wall time: 38.9 s


* ###  From the $3150^3$ box, we extract $31^3$ sub-boxes with the size, $L = 100 h^{-1}$Mpc

> For each subboxes, we will take top 2000 most massive halos.

In [9]:
lsys = 3150.0
mlength = 100.0
mdiv = 31

In [10]:
subdf = halodf\
.withColumn('ixdiv',F.floor(halodf.px/mlength))\
.withColumn('iydiv',F.floor(halodf.py/mlength))\
.withColumn('izdiv',F.floor(halodf.pz/mlength))

In [11]:
subdf = subdf\
.withColumn('ixdiv',F.when(subdf.ixdiv == -1, 0).otherwise(subdf.ixdiv))\
.withColumn('iydiv',F.when(subdf.iydiv == -1, 0).otherwise(subdf.iydiv))\
.withColumn('izdiv',F.when(subdf.izdiv == -1, 0).otherwise(subdf.izdiv))

In [12]:
subdf.cache()

DataFrame[haloid: int, px: float, py: float, pz: float, halomass: float, ixdiv: bigint, iydiv: bigint, izdiv: bigint]

In [13]:
subdf.show(10,truncate=True)

+------+---------+---------+---------+-------------+-----+-----+-----+
|haloid|       px|       py|       pz|     halomass|ixdiv|iydiv|izdiv|
+------+---------+---------+---------+-------------+-----+-----+-----+
|     0|106.23875|2820.2603|310.53067|3.29161999E14|    1|   28|    3|
|     1|1015.0091| 3070.103|2687.5447|   5.79631E14|   10|   30|   26|
|     2|1150.7571| 656.3275|195.96417| 7.4869997E14|   11|    6|    1|
|     3|2745.5166|2020.5542|839.78326|1.55997994E14|   27|   20|    8|
|     4|1129.6155|1974.3558|1253.1699|3.26915999E14|   11|   19|   12|
|     5|589.67566|2289.6702|2542.1628|2.77195998E14|    5|   22|   25|
|     6|1486.9604| 2534.887| 2683.816|1.08108999E14|   14|   25|   26|
|     7|3036.3958|13.892702|2686.4326|1.14171002E14|   30|    0|   26|
|     8|1077.6719|230.39247|2675.5627| 7.3316702E14|   10|    2|   26|
|     9|1666.9918|658.03186| 472.5904|5.46761014E14|   16|    6|    4|
+------+---------+---------+---------+-------------+-----+-----+-----+
only s

In [14]:
%%time
ixmin = subdf.agg({"ixdiv":"min"}).collect()[0][0]
iymin = subdf.agg({"iydiv":"min"}).collect()[0][0]
izmin = subdf.agg({"izdiv":"min"}).collect()[0][0]
print "idiv code min = ", ixmin," ",iymin," ",izmin

idiv code min =  0   0   0
CPU times: user 17.3 ms, sys: 8.74 ms, total: 26 ms
Wall time: 59.5 s


In [15]:
%%time
ixmax = subdf.agg({"ixdiv":"max"}).collect()[0][0]
iymax = subdf.agg({"iydiv":"max"}).collect()[0][0]
izmax = subdf.agg({"izdiv":"max"}).collect()[0][0]
print "idiv code min = ", ixmax," ",iymax," ",izmax

idiv code min =  31   31   31
CPU times: user 6.49 ms, sys: 2.9 ms, total: 9.39 ms
Wall time: 1.15 s


In [16]:
print 31**3

29791


In [17]:
print range(31)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]


In [18]:
# the cell id_s, ix, iy, iz, from 0 to 30
idmax = 31 # idmax=31 == [0, ... ,30]
redf = pd.DataFrame()
icounter = 0
numhalo = 2000 # how many halos for the output
for ix in range(idmax):
    for iy in range(idmax):
        for iz in range(idmax):
            gc.collect()
            tmppd = subdf.filter((F.col('ixdiv') ==ix) & (F.col('iydiv') ==iy) & (F.col('izdiv') ==iz))\
            .select('haloid','px','py','pz','halomass').toPandas()
            tmppd = tmppd.sort_values(by='halomass',ascending=False)[0:numhalo] #sort the DF based on halomass
            tmppd['isect'] = np.full(numhalo,icounter) #id for each section 
            #print "ix, iy, iz, numdata : ",ix," ",iy," ", iz," ",len(tmppd.index)
            #print tmppd.head(3)
            redf = pd.concat([redf,tmppd])
            icounter = icounter + 1
            
            
            

In [19]:
#tmppd.sort_values(by='halomass',ascending=False)[0:5]

In [20]:
redf = redf.reset_index(drop=True)
print len(redf.index)

59582000


* ###  write the result pandas-dataframe `redf` to a parquet file

In [21]:
import pyarrow as pa
import pyarrow.parquet as pq

In [22]:
pq.write_table(pa.Table.from_pandas(redf), 'hrdata.parquet.snappy', compression='snappy')